# Training Loop Example

This notebook demonstrates how to implement a custom trainer class to simplify model training setup.

In [1]:
%load_ext tensorboard
%tensorboard --logdir logs --port 6006 --bind_all

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import time
import numpy as np
import datetime

# --- Setup (Hyperparameters, Model, Data, etc.) ---

# Hyperparameters
epochs = 10
learning_rate = 0.001
batch_size = 64
num_features = 10
num_classes = 5

# Define a simple neural network
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.layer1 = nn.Linear(num_features, 128)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(128, num_classes)

    def forward(self, x):
        return self.layer2(self.relu(self.layer1(x)))

# Create dummy data and DataLoaders
# In a real scenario, you would load your actual dataset here
X_train = torch.randn(1000, num_features)
y_train = torch.randint(0, num_classes, (1000,))
X_val = torch.randn(200, num_features)
y_val = torch.randint(0, num_classes, (200,))

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)

# Instantiate model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

2025-10-13 03:39:34.710479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760326774.722870   14534 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760326774.726738   14534 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-10-13 03:39:34.738700: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
class Trainer:

    def __init__(
        self, model, optimizer, criterion, train_dataloader, device, val_dataloader=None, log_dir=None
    ):
        self._model = model
        self._optimizer = optimizer
        self._criterion = criterion
        self._train_dataloader = train_dataloader
        self._val_dataloader = val_dataloader
        self._device = device
        self._log_dir = log_dir

    def _train_step(self, epoch, epochs, writer):
        """Training phase in an epoch."""
        self._model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
    
        train_pbar = tqdm(self._train_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for i, (inputs, labels) in enumerate(train_pbar):
            inputs, labels = inputs.to(self._device), labels.to(self._device)
            
            # Forward pass
            outputs = self._model(inputs)
            loss = self._criterion(outputs, labels)
            
            # Backward and optimize
            optimizer.zero_grad()  # clear previous gradients
            loss.backward()        # compute the gradient of the loss
            optimizer.step()       # update the model parameters
            
            # Logging batch-level metrics
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
    
            # Update tqdm progress bar description
            train_pbar.set_postfix({'loss': loss.item()})
            
            # Log batch loss to TensorBoard
            global_step = epoch * len(train_loader) + i
            writer.add_scalar('Loss/train_batch', loss.item(), global_step)
        
        avg_train_loss = train_loss / len(train_loader)
        train_accuracy = 100 * train_correct / train_total
        print(f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")
        
        writer.add_scalar('Loss/train_epoch', avg_train_loss, epoch)
        writer.add_scalar('Accuracy/train_epoch', train_accuracy, epoch)

    def _validation_step(self, epoch, epochs, writer):
        """Validation phase in an epoch."""
        self._model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_pbar = tqdm(self._val_dataloader, desc=f"Epoch {epoch+1}/{epochs} [Val]")
            for inputs, labels in val_pbar:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = self._criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
                val_pbar.set_postfix({'loss': loss.item()})
        
        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = 100 * val_correct / val_total
        print(f"Validation Loss: {avg_val_loss:.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        writer.add_scalar('Loss/validation_epoch', avg_val_loss, epoch)
        writer.add_scalar('Accuracy/validation_epoch', val_accuracy, epoch)

    def train(self, epochs):
        """Main method to start the model training."""
        print(f"Starting training on {self._device}...")
        log_dir = self._log_dir
        if log_dir is None:
            log_dir = f"logs/fit/{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}"
        writer = SummaryWriter(log_dir)
        print(f"Saving logs to {log_dir}")

        train_start_t = time.time()
        for epoch in range(epochs):
            # --- Training Phase ---
            epoch_start_t = time.time()
            self._train_step(epoch, epochs, writer)
            train_epoch_stop_t = time.time()
            
            # --- Validation Phase ---
            if self._val_dataloader:
                self._validation_step(epoch, epochs, writer)
            epoch_end_t = time.time()
        
            # --- End-of-Epoch Logging ---
            train_duration = train_epoch_stop_t - epoch_start_t
            val_duration = epoch_end_t - train_epoch_stop_t
            epoch_duration = epoch_end_t - epoch_start_t
            val_log = f"Validation Time: {val_duration:.2f}s | " if self._val_dataloader else ""
            print(
                f"Epoch {epoch+1}/{epochs} | "
                f"Train Time: {train_duration:.2f}s | {val_log}"
                f"Epoch Time: {epoch_duration:.2f}s")
            writer.add_scalar('Learning_Rate', self._optimizer.param_groups[0]['lr'], epoch)

        total_train_duration = time.time() - train_start_t
        print(f"Total Training Time: {str(datetime.timedelta(seconds=total_train_duration))} | {total_train_duration:.2f}s")
        print("Training complete.")
              
        writer.close()

In [4]:
trainer = Trainer(model, optimizer, criterion, train_loader, device, val_dataloader=val_loader)
trainer.train(30)

Starting training on cuda...
Saving logs to logs/fit/20251013-033937


Epoch 1/30 [Train]: 100%|██████████| 16/16 [00:00<00:00, 147.56it/s, loss=1.61]


Train Loss: 1.6274 | Train Accuracy: 21.20%


Epoch 1/30 [Val]: 100%|██████████| 4/4 [00:00<00:00, 796.45it/s, loss=1.65]


Validation Loss: 1.6196 | Val Accuracy: 20.00%
Epoch 1/30 | Train Time: 0.11s | Validation Time: 0.01s | Epoch Time: 0.12s


...

Validation Loss: 1.6396 | Val Accuracy: 21.00%
Epoch 29/30 | Train Time: 0.03s | Validation Time: 0.01s | Epoch Time: 0.03s


Epoch 30/30 [Train]: 100%|██████████| 16/16 [00:00<00:00, 580.44it/s, loss=1.41]


Train Loss: 1.4857 | Train Accuracy: 35.30%


Epoch 30/30 [Val]: 100%|██████████| 4/4 [00:00<00:00, 1314.11it/s, loss=1.58]

Validation Loss: 1.6383 | Val Accuracy: 20.00%
Epoch 30/30 | Train Time: 0.03s | Validation Time: 0.00s | Epoch Time: 0.03s
Total Training Time: 0:00:01.045402 | 1.05s
Training complete.
